This notebook is used to export data from a netCDF file to a csv file (which is the easiest to import into geoscene3d).

Neil Symington
neil.symington@ga.gov.au

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4
import os, subprocess
import re
import numpy as np
import fiona
from shapely.geometry import Polygon, MultiPoint
from geophys_utils._netcdf_point_utils import NetCDFPointUtils
from hydrogeol_utils import spatial_functions
import rasterio
import pandas as pd
import time

In [2]:
import importlib
importlib.reload(spatial_functions)

<module 'hydrogeol_utils.spatial_functions' from 'c:\\users\\pcuser\\onedrive\\github\\hydrogeol_utils\\hydrogeol_utils\\spatial_functions.py'>

In [3]:
# Import the data

#infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\nc\HowardsEast_wb_inversion.nc"

infile = "http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/axi547/aem/AusAEM_Year1_NT_Final_CND.nc"

lci_dat = netCDF4.Dataset(infile)

In [4]:
lci_dat#['data_residual'][:].shape

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    title: Dataset read from ASEG-GDF file AusAEM_Year1_NT_Final_CND.dat
    Conventions: CF-1.6,ACDD-1.3
    featureType: trajectory
    geospatial_vertical_min: 0.06
    geospatial_vertical_max: 0.06
    geospatial_vertical_units: m
    geospatial_vertical_resolution: point
    geospatial_vertical_positive: up
    history: Converted from ASEG-GDF file AusAEM_Year1_NT_Final_CND.dat using definitions file AusAEM_Year1_NT_Final_CND.dfn
    date_created: 2019-09-20T11:08:31.121466
    geospatial_east_resolution: point
    geospatial_north_resolution: point
    geospatial_lon_min: 131.989990234375
    geospatial_lon_max: 138.09056091308594
    geospatial_lon_units: degrees East
    geospatial_lat_min: -24.06076431274414
    geospatial_lat_max: -15.825227737426758
    geospatial_lat_units: degrees North
    geospatial_bounds: POLYGON((135.1500 -24.0608, 134.1653 -24.0585, 132.3088 -24.0146, 132.09

In [5]:
# Bring in the spatial mask

shp_file = r"C:/Users/PCUser/Desktop/SSC_data/vector/PWC_AEM_polygons.shp"

shape = fiona.open(shp_file)

shp = shape.next()
shp2 = shape.next()
shp3 = shape.next()
shp4 = shape.next()
shp5 = shape.next()
shp6 = shape.next()
shp7 = shape.next()

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:7: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  import sys
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:8: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:9: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  if __name__ == '__main__':
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:10: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PCUser\Anaconda3\envs\hydro

In [6]:
shp7

{'type': 'Feature',
 'id': '6',
 'properties': OrderedDict([('id', None), ('Community', 'alice')]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(138659.057406227, 7561409.007124317),
    (519267.8025343473, 7551790.735586783),
    (527512.0352808047, 7414386.856479159),
    (135910.9798240745, 7421257.050434539),
    (138659.057406227, 7561409.007124317)]]}}

In [7]:
poly = Polygon(shp7['geometry']['coordinates'][0])

poly.bounds

(135910.9798240745, 7414386.856479159, 527512.0352808047, 7561409.007124317)

In [8]:
lci_dat['easting']

<class 'netCDF4._netCDF4.Variable'>
float32 easting(point)
    projection_name: GDA94/53S
    aseg_gdf_format: E10.2
    units: m
    long_name: UTM Easting (x) ordinate
    _FillValue: -99999.99
    _ChunkSizes: 1024
unlimited dimensions: 
current shape = (2346636,)
filling off

In [9]:
cond_point_utils = NetCDFPointUtils(lci_dat)
#utm_wkt = 'PROJCS["GDA94 / MGA zone 52",GEOGCS["GDA94",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6283"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4283"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",129],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",10000000],AUTHORITY["EPSG","28352"],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
utm_wkt = 'PROJCS["GDA94 / MGA zone 53",GEOGCS["GDA94",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6283"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4283"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",135],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",10000000],AUTHORITY["EPSG","28353"],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [10]:
spatial_mask = cond_point_utils.get_spatial_mask(bounds = poly.bounds,
                                                 bounds_wkt = utm_wkt)

In [ ]:
lci_dat

In [12]:
import importlib
importlib.reload(netcdf2aseg_gdf_converter)

NameError: name 'netcdf2aseg_gdf_converter' is not defined

In [13]:
# testing

#
from geophys_utils.netcdf_converter import netcdf2aseg_gdf_converter

nc2asegconv = netcdf2aseg_gdf_converter.NetCDF2ASEGGDFConverter(infile,
                                                        crs_string='EPSG:4283',
                                                        )

In [14]:
dat_out_path = r"C:\Users\PCUser\Documents\AEM\AusAEM_Year1_Tranche1_Final_CND.dat"
dfn_out_path = r"C:\Users\PCUser\Documents\AEM\AusAEM_Year1_Tranche1_Final_CND.dfn"

nc2asegconv.convert2aseg_gdf(dat_out_path, dfn_out_path,
                            spatial_mask)

line
I
I
flight
I
I
fiducial
F
proj_cgg
I
proj_client
I
date
I
time_local
F
time_utc
F
gps_time
F
tx_bearing
F
latitude
F
longitude
F
easting
F
northing
F
gps_elevation
F
tx_elevation
F
lidar
F
radalt
F
dtm
F
mag
F
tx_height
F
cond_xz


RuntimeError: NetCDF: Access failure

In [ ]:
lci_dat['cond_xz'][spatial_mask
                  ]

In [ ]:
cond = lci_dat['conductivity']#[spatial_mask]
easting = lci_dat['easting']#[spatial_mask]
northing = lci_dat['northing']#[spatial_mask]
elevation = lci_dat['elevation']#[spatial_mask]
layer_top_depth = lci_dat['layer_top_depth']#[spatial_mask]
doi = lci_dat['depth_of_investigation']#[spatial_mask]
line = lci_dat['line'][lci_dat['line_index']]#[spatial_mask]]
fid = lci_dat['sequence']#[spatial_mask]

In [ ]:
df = pd.DataFrame(data = {'fiducial': fid,
                          'line': line,
                         'easting': easting,
                         'northing': northing,
                         'elevation': elevation,
                         'doi': doi})

In [ ]:
for i in range(30):
    df['Cond_' + str(i+1)] = cond[:,i]

In [ ]:
for i in range(30):
    df['Layer_top_depth_' + str(i+1)] = layer_top_depth[:,i]

In [ ]:
#outfile = r"C:\Users\PCUser\Desktop\EK_data\Geoscene3d\datasets\AEM\VicHighway_AEM.csv"
outfile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\G3D_files\HowardsEast_AEM.csv"
df.to_csv(outfile,  index=False)

In [ ]:
lci_dat['crs']